In [9]:
import re
import json
import time
import random
from lxml import etree
# import pymysql
import requests
from lxml import etree
import requests
from pymongo import MongoClient
import gridfs
import os
poem=[]
poem_name=[]
poem_author=[]
author_year=[] 
poem=[]
poem_name=[]
poem_author=[]
author_year=[]

In [18]:
class Gushici():

    def __init__(self):
        self.base_url = "https://so.gushiwen.cn/shiwens/"
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.183 Safari/537.36'
        }

    def url_list(self):
        urls=[]
        sec_urls=[]
        shi=[]
        ci=[]
        qu=[]
        wen=[]
        r = requests.get(self.base_url, headers=self.headers)
        r.encoding = "UTF-8"
        content = etree.HTML(r.content)

        type=content.xpath('//*[@id="html"]/body/div[2]/div[1]/div[1]/div[5]/div[2]//@href')
        for i in type:
            i=list(i)
            i.insert(22, 'page= &')
            i.insert(0, 'https://so.gushiwen.cn')
            i="".join(i)
            urls.append(i)
        for i in urls:
            for page in range(1,11):
                page=str(page)
                url_second=i.replace(' ',page)
                r = requests.get(url_second, headers=self.headers)
                r.encoding = "UTF-8"
                content = etree.HTML(r.content)
                sec_url=content.xpath('//*[@class="sons"]/div[1]/p[1]//@href')
                # sec_urls.append(page)
                sec_urls.append(sec_url)
        

        a=0
        for i in sec_urls:
            a=a+1
            i=str(i)
            i=i.replace("'","")
            i=i.split(",")
            for j in range(len(i)):
            # print(i[0])
                i[j]=i[j].replace("[","").replace("]","").replace(" ","")
                i[j]='https://so.gushiwen.cn'+i[j]
                if(a>=1 and a<=10):
                    shi.append(i[j])
                
                elif(a>=11 and a<=20):
                    ci.append(i[j])
                    
                elif(a>=21 and a<=30):
                    qu.append(i[j])
                    
                else:
                    wen.append(i[j])
           
        return shi,ci,qu,wen


    def get_main_inf(self,list1):
        for i in list1:
            r = requests.get(i, headers=self.headers)
            r.encoding = "UTF-8"
            content = etree.HTML(r.content)
            poem_0=(i[31:43])
            p_name = content.xpath("//div[@class='sons']//div//h1//text()")
            p_author = content.xpath("//div[@id='sonsyuanwen']//div//p//a[1]//text()")
            poem_1=content.xpath("//*[@id='contson"+poem_0+"']//text()")
            a_year=content.xpath("//div[@id='sonsyuanwen']//div//p//a[2]//text()")
            if p_author=='':
                p_author='佚名'
                a_year='未知'
            poem.append(poem_1)
            # print(poem_1)
            # break
            poem_name.append(p_name)
            poem_author.append(p_author)
            author_year.append(a_year)
                
                            
        for i in range(len(poem)):
            poem[i]=str(poem[i])
            poem[i]=poem[i].replace("\\n","").replace("', '","").replace("'","").replace("[","").replace("]","").replace("\u3000","")
            # print(poem)
        for i in range(len(author_year)):
            author_year[i]=str(author_year[i])
            author_year[i]=author_year[i].replace("'〔","").replace("〕'","").replace("[","").replace("]","").replace("\u3000","")
            # print(author_year[i])
        for i in range(len(poem_author)):
            poem_author[i]=str(poem_author[i])
            poem_author[i]=poem_author[i].replace("'","").replace("[","").replace("]","").replace("\u3000","") 
            # print(poem_author[i])
        for i in range(len(poem_name)):
            poem_name[i]=str(poem_name[i])
            poem_name[i]=poem_name[i].replace("[","").replace("]","").replace("'","").replace("\u3000","")

        # print(shi)
        # print(ci)
        # print(qu)
        # print(wen)

In [21]:
test=Gushici()
shi,ci,qu,wen=test.url_list()
test.get_main_inf(wen)

In [23]:
list1=['作者名','作者朝代','文言文名','内容']
poem_list=[]
for i in range(len(poem)):
          list_poem=[]
          list_poem.clear()
          list_poem.append(poem_author[i])
          list_poem.append(author_year[i])
          list_poem.append(poem_name[i])
          list_poem.append(poem[i])
          # print(list_poem)
          d=zip(list1,list_poem)
          # print(dict(d))
          # print(type(d))
          poem_list.append(dict(d))
# print(poem_list)

import pymongo
from pymongo import MongoClient
import gridfs
import os
client=MongoClient('localhost',27017)

def save_data():
        gushi=Gushici()
        gushi.url_list()
        print(poem_list)
        try:
            client = pymongo.MongoClient(host='127.0.0.1')
            db =client['poetry']
            collections = db.wenyanxhuy
            for data in poem_list:
          #       print(data)
                collections.insert_one(data)
            print("插入成功！")
        except Exception as e:
            print(e)

save_data()

[{'作者名': '苏轼', '作者朝代': '宋代', '文言文名': '水调歌头·明月几时有', '内容': '丙辰中秋，欢饮达旦，大醉，作此篇，兼怀子由。明月几时有？把酒问青天。不知天上宫阙，今夕是何年。我欲乘风归去，又恐琼楼玉宇，高处不胜寒。起舞弄清影，何似在人间。(何似 一作：何时；又恐 一作：惟 / 唯恐)转朱阁，低绮户，照无眠。不应有恨，何事长向别时圆？人有悲欢离合，月有阴晴圆缺，此事古难全。但愿人长久，千里共婵娟。(长向 一作：偏向)'}, {'作者名': '苏轼', '作者朝代': '宋代', '文言文名': '念奴娇·赤壁怀古', '内容': '大江东去，浪淘尽，千古风流人物。故垒西边，人道是，三国周郎赤壁。乱石穿空，惊涛拍岸，卷起千堆雪。(穿空 一作：崩云)江山如画，一时多少豪杰。遥想公瑾当年，小乔初嫁了，雄姿英发。羽扇纶巾，谈笑间，樯橹灰飞烟灭。(樯橹 一作：强虏)故国神游，多情应笑我，早生华发。人生如梦，一尊还酹江月。(人生 一作：人间；尊 同：樽) '}, {'作者名': '毛泽东', '作者朝代': '近现代', '文言文名': '沁园春·雪', '内容': '北国风光，千里冰封，万里雪飘。望长城内外，惟余莽莽；大河上下，顿失滔滔。(余 通：馀)山舞银蛇，原驰蜡象，欲与天公试比高。(原驰 原作：原驱)须晴日，看红装素裹，分外妖娆。(红装 一作：银装)江山如此多娇，引无数英雄竞折腰。惜秦皇汉武，略输文采；唐宗宋祖，稍逊风骚。一代天骄，成吉思汗，只识弯弓射大雕。俱往矣，数风流人物，还看今朝。'}, {'作者名': '岳飞', '作者朝代': '宋代', '文言文名': '满江红·写怀', '内容': '怒发冲冠，凭栏处、潇潇雨歇。抬望眼，仰天长啸，壮怀激烈。三十功名尘与土，八千里路云和月。莫等闲，白了少年头，空悲切！(栏 通：阑)靖康耻，犹未雪。臣子恨，何时灭！驾长车，踏破贺兰山缺。壮志饥餐胡虏肉，笑谈渴饮匈奴血。待从头、收拾旧山河，朝天阙。(壮志 一作：壮士；山缺 一作：山阙) '}, {'作者名': '苏轼', '作者朝代': '宋代', '文言文名': '江城子·乙卯正月二十日夜记梦', '内容': '十年生死两茫茫，不思量，自难忘。千里孤坟，无处话凄凉。纵使相逢应不识，尘满面，鬓如霜。夜来幽梦忽还乡，小轩窗，正梳妆。相顾无言，惟有泪